# Example 1: Formula Creation

In [ ]:
from mlisplacement.synthdoc.structs.azure import CHARGES as AZURE_CHARGES #TODO: implement AZURE same way as AIRPORTS
from mlisplacement.synthdoc.structs.azure import VARIABLES as AZURE_VARIABLES
from mlisplacement.structs.airports import AirportCharges, AirportVariables 
from mlisplacement.synthdoc.function_creation import create_charge_func, generate_output_structure

First instantiate the charges and variables

In [2]:
charges = AirportCharges()
variables = AirportVariables()

The following function creates a charge-function for a chosen charge.

Note they accept dicts.

In [3]:
create_charge_func("security_fee", charges.to_dict(), variables.to_dict())

("if aircraft_weight in [5000.0, 58752.6), charge is 0 x aircraft_weight; if aircraft_weight in [58752.6, 393870.7), charge is 1 x aircraft_weight; if aircraft_weight in [393870.7, 400000.0), charge is 2 x aircraft_weight The variable 'aircraft_weight' is measured in kg.",
 "def compute_charge(aircraft_weight):\n    if 5000.0 <= aircraft_weight < 58752.6: return 0 * aircraft_weight\n    elif 58752.6 <= aircraft_weight < 393870.7: return 1 * aircraft_weight\n    elif 393870.7 <= aircraft_weight < 400000.0: return 2 * aircraft_weight\n    else: raise ValueError(f'aircraft_weight is out of expected range')",
 ['aircraft_weight'])

The following function creates a charge-function for all charges in the given structure.

In [4]:
charges_dict = generate_output_structure(charges.to_dict(), variables.to_dict())

In [5]:
charges_dict

{'security_fee': {'description': "if plane weight in [5000.0, 115991.4), charge is 0 x plane weight; if plane weight in [115991.4, 400000.0), charge is 1 x plane weight The variable 'plane weight' is measured in kg.",
  'code': "def compute_charge(aircraft_weight):\n    if 5000.0 <= aircraft_weight < 115991.4: return 0 * aircraft_weight\n    elif 115991.4 <= aircraft_weight < 400000.0: return 1 * aircraft_weight\n    else: raise ValueError(f'aircraft_weight is out of expected range')",
  'variables_used': ['aircraft_weight'],
  'synonyms': ['security charge', 'safety fee'],
  'charge_description': 'A fee charged for security measures at the airport.'},
 'landing_fee': {'description': 'landing_fee conditions by aircraft category and traveler count:\n\nfor passenger the following charges apply:\n  - if traveler count in [277, 500), charge is 0 x traveler count\n  - if traveler count in [10, 277), charge is 1 x traveler count\nfor cargo the following charges apply:\n  - if traveler count 

# Example 2: Generate Document from Structured Charges

In [6]:
from mlisplacement.synthdoc.llm_gen import LLMClient, generate_document_from_json

In [7]:
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"  # Unrestricted example. This is small and wont make a nice document...

In [8]:
# from mlisplacement.utils.credentials_utils import set_hf_token
# set_hf_token()
# MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct" #restricted example. needs the HF token set in the environment variable HF_TOKEN

In [9]:
model = LLMClient(
    model_name=MODEL_NAME, # Example model, replace with your choice/one for which you have access
)

Device set to use cuda:0


In [10]:
charges_dict = generate_output_structure(charges.to_dict(), variables.to_dict())

In [11]:
doc, answers, names = generate_document_from_json(
    client=model,
    charges_dict=charges_dict)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [12]:
print(doc)

# Serenia Aeronautica Charging Policy Document

<table>
  <tr>
    <th>Section</th>
    <th>Heading</th>
    <th>Definition</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>1</td>
    <td>Charging Policy</td>
    <td>A comprehensive policy outlining the fees and charges for various aviation services at Serenia Aeronautica Airport.</td>
    <td>A detailed explanation of our fee structure and charging policies for airport services.</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Pricing Categories</td>
    <td>A categorization of fees for different types of airport services.</td

## Airport Security Fee Policy

<table border="0">
  <tr>
    <th>Charge Category</th>
    <td>Traveler Count</td>
    <td>Charge Amount</td>
  </tr>
  <tr>
    <td>0-282</td>
    <td>1</td>
    <td>0</td>
  </tr>
  <tr>
    <td>283-500</td>
    <td>2</td>
    <td>0</td>
  </tr>
  <tr>
    <td>501-115</td>
    <td>3</td>
    <td>0</td>
  </tr>
  <tr>
    <td>116-282</td>
    <td>1</td>
    <td>0</td>
  </tr>
  <t

In [14]:
print("-" * 100)
for name, answer in zip(names, answers):
    print(f"Answer for {name}:")
    print(answer)
    print("-" * 100)

----------------------------------------------------------------------------------------------------
Answer for security_fee:
def compute_charge(passenger_count):
    if 282 <= passenger_count < 500: return 0 * passenger_count
    elif 115 <= passenger_count < 282: return 1 * passenger_count
    elif 10 <= passenger_count < 115: return 2 * passenger_count
    else: raise ValueError(f'passenger_count is out of expected range')
----------------------------------------------------------------------------------------------------
Answer for landing_fee:
def compute_charge(flight_type, fuel_consumption):
    if flight_type == 'domestic' and 500.0 <= fuel_consumption < 620.2: return 0 * fuel_consumption * 1.0
    elif flight_type == 'domestic' and 620.2 <= fuel_consumption < 4447.9: return 1 * fuel_consumption * 1.0
    elif flight_type == 'domestic' and 4447.9 <= fuel_consumption < 5000.0: return 2 * fuel_consumption * 1.0
    elif flight_type == 'international' and 500.0 <= fuel_consumption